# Project: Investigate a Dataset - TMDb Movie Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> This data set contains information about 10,000 movies collected from The Movie Database (TMDb), including user ratings and revenue. 
>- Certain columns, like ‘cast’ and ‘genres’, contain multiple values separated by pipe (|) characters. 
>- There are some odd characters in the ‘cast’ column. Don’t worry about cleaning them. You can leave them as is. 
>- The final two columns ending with “_adj” show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time.. 


### Question(s) for Analysis
1. What is the Trend Analysis of the Dataset
2. Which Director has the most film produced
3. Which movie has cost savings in excess of $50 million

### importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#magic word use to show plot within the document
%matplotlib inline
#set maximum rows to display to 20000
#pd.options.display.max_rows = 20000

<a id='wrangling'></a>
## Data Wrangling

#### Loading the dataset and checking general information about the dataset

In [ ]:
mov_df = pd.read_csv('tmdb-movies.csv')

In [ ]:
#inspecting a few of the dataset
mov_df.head(2)

In [ ]:
#checking the dataset to information
mov_df.info()

#### There are a lot of duplicates that we need to explore, the release date need to be change from string to datetime, there are columns that will not impact our analysis, such as tagline, homepage, keywords, as such they need to be dropped

In [ ]:
mov_df.nunique()

#### though there are 10866 samples, the unique record for movie title is 10,571, which suggest that thre are duplicates and since this samples are about movies, it makes sence to work with the unique records in our analysis. we will also check to see why revenue is 4702, are there movies that did not generate revenues?


### Data Cleaning
1. remove duplicate titles
2. The following column will be remove as they will not impact on the analysis
    - id
    - imdb_id
    - homepage
    - tagline
    - keywords
    - overview
3. release date should be of the datetime data type
4. we will check for duplicates and remove them
5. ensure the data type are right for analysis 

#### check for dupicates in the dataset and how many are in the dataset

In [ ]:
#checking for duplicates and the count
dup_title = mov_df[mov_df['original_title'].duplicated() ==True]
dup_title['original_title'].count()

#### removing the duplicates

In [ ]:
mov_df.drop(dup_title['original_title'].index, axis=0, inplace=True)

In [ ]:
mov_df.info()

#### removing columns not needed in our analysis

In [ ]:
#Columns to drop
col_to_drop = ['id', 'imdb_id', 'homepage', 'tagline', 'keywords', 'overview' ]

In [ ]:
#Droping the columns
mov_df.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
#confirming that the columns were drpped
mov_df.info()

#### There appear to still exist missing data in cast, director, genres and production companies, but these missing data can be ignored

#### converting the release date from string to datetime data type

In [ ]:
#converting the release date to datetime
mov_df['release_date'] = pd.to_datetime(mov_df['release_date'])

In [ ]:
#checking the data information
mov_df.info()

In [ ]:
mov_df.duplicated().sum()

### The dataset is now free of duplicates

#### check for movie that had no budget and zerio revenue

In [ ]:
#check to ensure we do not have 0 revenue or in the adjusted revenue
mov_df.query('revenue == 0 and revenue_adj ==0')

### We will also remove these data because they did not generate any revenue

In [ ]:
zero_rev_mov_index = mov_df.query('revenue == 0 and revenue_adj ==0').index

In [ ]:
zero_rev_mov_index

In [ ]:
mov_df.drop(zero_rev_mov_index, axis=0, inplace=True)

In [ ]:
mov_df.info()

In [ ]:
mov_df.to_csv('clean_mov_df', index=False)

### This dataset is now clean and ready for analysis

<a id='eda'></a>
## Exploratory Data Analysis



### Research Question 1 What is the Trend Analysis of the dataset

In [ ]:
#importing the clean movie dataset
c_mov_df = pd.read_csv('clean_mov_df')

In [ ]:
#Plotting number of film produced by year
c_mov_df.groupby('release_year')['original_title'].count().plot(kind='line', figsize=(15,15))
plt.title('Number of Movies release from 1960 to 2015')
plt.xlabel('Release Year')
plt.ylabel('Number of Movies')
plt.legend();

### The chart above showed that movies production started to increase in the late 1970s and has continue ever since

In [ ]:
#sampling data from 1980
mov_data_80_to_15 = c_mov_df.query('release_year >= 1980')

In [ ]:
mov_data_80_to_15.groupby('release_year')['original_title'].count().plot(kind='bar', figsize=(15,8))
plt.title('Number of Movies release from 1980 to 2015')
plt.xlabel('Release Year')
plt.ylabel('Number of Movies')
plt.legend();

### The Chart above indicated that though movie production had been on the rise, there is a decline in 2012, production picked up in 2013 and from 2013, production has seen decline in 2014 and 2015

In [ ]:
rev = mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].max().revenue 
bug = mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].max().budget
print(rev-bug)
mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].max()

## The highest movie of all time with 9767 votes
- Title - ã‚¢ãƒƒãƒ—ãƒ«ã‚·ãƒ¼ãƒ‰
- \$2,781,505,847 in Revenue
- \$2,356,505,847 in profit


In [ ]:
rev = mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].min().revenue 
bug = mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].min().budget
print(rev-bug)
mov_data_80_to_15[['original_title','release_year','revenue', 'vote_count','budget']].min()

## The lowest movie of all time with 10 votes
- Title - 71
- \$2 in Revenue
- \$2 in profit

### Research Question 2  Director with most film production

In [ ]:
#retrieving the director and title column
dir_list_df = mov_data_80_to_15.loc[:,['director', 'original_title']]

In [ ]:
#Counting number of movie produced and sorting in descending order
top_dir = dir_list_df.groupby(['director']).count().sort_values(by='original_title',ascending=False)

In [ ]:
top_dir.query('original_title > 10')

In [ ]:
top_dir.query('original_title > 10').plot(kind='bar', figsize=(15,15))
plt.title('Number of Movies Produce by Director')
plt.xlabel('Director')
plt.ylabel('Number of Movies')
plt.legend();

#### from the graph above we can see that Steven Spielberg produced more movies than the rest of the directors

### Research Question 3  Movie with cost savings in excess of \$50,000,000

In [ ]:
#Adding a new column to the dataset for cost_savings
mov_data_80_to_15['cost_savings'] = mov_data_80_to_15['budget_adj'] - mov_data_80_to_15['budget']

In [ ]:
#Sorting the dataset in descending order
mov_with_cs = mov_data_80_to_15[['original_title', 'cost_savings']].sort_values(by='cost_savings', ascending=False)

In [ ]:
#Removing data that have less than $50,000,000 in cost savings
mov_with_pcs = mov_with_cs.query('cost_savings > 50000000')

In [ ]:
mov_with_pcs.head()

In [ ]:
#PLotting the result
plt.figure(figsize=(15,15))

plt.pie(mov_with_pcs.cost_savings, labels=mov_with_pcs.original_title, autopct='%1.1f%%')
plt.title('Movie with Cost savings');

#### Superman II is the movie with the highest cost savings before 2016

<a id='conclusions'></a>
## Conclusions

> #### Result
>
> - The movie industry had seen an upward trend from late 1970, though a decline had set in from 2013, I was unable to determine id the decline continued beyond 2015 because the dataset end in the year 2015.
>
> - The highest movie of all time before 2016 both in revenue and votes is "ã‚¢ãƒƒãƒ—ãƒ«ã‚·ãƒ¼ãƒ‰" while 71 is the lowest.
>
> - Before 2016, Steven Spielberg	had director 23 movies, followed by Clint Eastwood, who had directed 22 movies, the third place goes to Ridley Scoot with 19 movies, while Ron Howard and Woody Allen came fourth with 18 movies
>
> - While comparing the budget with the adjusted budget, the top cost saving movie is Supermann II
>
> #### Limitation
>
> - There are no record from 2016 to date, as such the decline experience in 2014 could not be analyse further
>
> - The dataset did not provided revenue earned by the director for each movie produced
>
> - The dataset did not provided revenue earned by the casts(actors) for the role in the movie


